In [1]:
#Based on https://github.com/aymericdamien/TensorFlow-Examples/blob/master/notebooks/2_BasicModels/linear_regression.ipynb
import tensorflow as tf
import numpy
import csv
rng = numpy.random

In [2]:
# Parameters
learning_rate = 0.01
training_epochs = 300
display_step = 50

In [3]:
#Training data
csv = numpy.genfromtxt("solves.csv", skip_header=1, delimiter=",")
train_difficulty = csv[:, 0]
train_features = csv[:, 1:]

In [4]:
#number of training samples
n_samples = train_difficulty.shape[0]

#number of features
feature_length = train_features.shape[1]

In [5]:
#Normalize all of the columns to be between 0 and 1 with a smooth distribution
#TODO: we need to save the maxVal so other folks in the model can normalize appropriately

for colNum in xrange(0, feature_length):
    col = train_features[:,colNum]
    maxVal = max(col)
    if maxVal == 0:
        maxVal = 1
    for index in xrange(0, len(col)):
        col[index] = col[index] / maxVal


In [6]:

# tf Graph Input
Difficulty = tf.placeholder("float")
Features = tf.placeholder("float", shape=(feature_length))

# Set model weights
W = tf.Variable(tf.zeros([feature_length]), name="weight")
b = tf.Variable(rng.randn(), name="bias")

In [7]:
# Construct a linear model
pred = tf.add(tf.mul(Features, W), b)

In [8]:
#TODO: calculate R2 and output it over time
# Mean squared error
with tf.name_scope("cost") as scope:
    cost = tf.reduce_sum(tf.pow(pred-Difficulty, 2))/(2*n_samples)
    cost_summ = tf.scalar_summary("cost", cost)
# Gradient descent
with tf.name_scope("optimize") as scope:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [9]:
#Summary ops to collect data
#The use of tensorboard is adapted from the example on https://www.tensorflow.org/versions/r0.7/how_tos/summaries_and_tensorboard/index.html
w_hist = tf.histogram_summary("weights", W)
b_hist = tf.histogram_summary("biases", b)
difficulty_hist = tf.histogram_summary("difficulty", Difficulty)

In [10]:
# Initializing the variables
init = tf.initialize_all_variables()

In [11]:
# Launch the graph
with tf.Session() as sess:
    
    merged = tf.merge_all_summaries()
    writer = tf.train.SummaryWriter("tmp/linear_logs", sess.graph_def)
    
    sess.run(init)

    # Fit all training data
    for epoch in range(training_epochs):
        for (features, difficulty) in zip(train_features, train_difficulty):
            sess.run(optimizer, feed_dict={Features: features, Difficulty: difficulty})

        #Display logs per epoch step
        if (epoch+1) % display_step == 0:
            result = sess.run([merged,cost], feed_dict={Features: features, Difficulty: difficulty})
            summary_str = result[0]
            c = result[1]
            writer.add_summary(summary_str, epoch)
            print "Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(c), \
                "W=", sess.run(W), "b=", sess.run(b)

    print "Optimization Finished!"
    training_cost = sess.run(cost, feed_dict={Features: features, Difficulty: difficulty})
    print "Training cost=", training_cost, "W=", sess.run(W), "b=", sess.run(b), '\n'


Epoch: 0050 cost= 0.006929512 W= [ 0.00215969  0.00216808  0.          0.          0.          0.          0.
  0.          0.00158258  0.00156512  0.00257801  0.00263026  0.00332769
  0.00351269  0.00450195  0.00459032  0.00033646  0.0003012   0.00021466
  0.00022114  0.00041621  0.00044062  0.          0.          0.00039225
  0.00037214  0.00019356  0.00019496  0.0001285   0.00012706  0.00020965
  0.0002089   0.00040046  0.00037575  0.00590786  0.0067174   0.00385812
  0.00430842  0.0038962   0.00455213  0.00146822  0.00144065  0.00080753
  0.00075361  0.00079815  0.00087841  0.00272689  0.00288739  0.00213027
  0.00216154  0.00225104  0.00245011  0.00369201  0.00095915  0.00651929
  0.00350325  0.00449189 -0.0001453   0.0055797   0.0063279   0.00058004
 -0.00336011  0.00099547 -0.0024995   0.00294938 -0.00162742  0.00272066
  0.00036354 -0.000378    0.00191732  0.00195985  0.00192128  0.00198255
 -0.01334671  0.00117908  0.00115726  0.00113866  0.00105449  0.00183845
  0.00200087  

In [ ]:
#TODO: output the model with bias, cost, and weights all zipped up with their names